# Importing Libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
import numpy as np


df = pd.read_csv("/kaggle/input/sicpreprocesseddataset/Synthetic_Financial_dataset.csv")

df

/tmp/ipykernel_36/4232924438.py:9: DtypeWarning: Columns (0,1,3,5,6,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/input/sicpreprocesseddataset/Synthetic_Financial_dataset.csv")


,transaction_id,step,type,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,1,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,False
1,2,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,False
2,3,1,TRANSFER,181.0,C1305486145,181.0,0.0,C553264065,0.0,0.0,True
3,4,1,CASH_OUT,181.0,C840083671,181.0,0.0,C38997010,21182.0,0.0,True
4,5,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...
6362619,1590651,156,PAYMENT,10221.33,C1331544292,17205.0,6983.67,M1373391563,0.0,0.0,False
6362620,1590652,156,PAYMENT,5364.42,C1284305343,20100.0,14735.58,M1218745950,0.0,0.0,False
6362621,1590653,156,CASH_IN,118042.97,C1776235854,52293.0,170335.97,C1401659879,198145.16,80102.2,False
6362622,1590654,156,CASH_OUT,696912.65,C1525067948,20585.0,0.0,C57107721,0.0,696912.65,False


# Load and Preprocess the Dataset

In [2]:
# Load the dataset
df = pd.read_csv("/kaggle/input/sicpreprocesseddataset/Synthetic_Financial_dataset.csv", low_memory=False)

# Convert 'step', 'amount', 'oldbalanceOrig', 'newbalanceOrig', 'oldbalanceDest', and 'newbalanceDest' to numeric
numeric_columns = ['step', 'amount', 'oldbalanceOrig', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Drop irrelevant columns
df = df.drop(['nameOrig', 'nameDest', 'transaction_id'], axis=1)

# One-Hot Encoding for 'type' column
df = pd.get_dummies(df, columns=['type'], drop_first=True)

In [3]:
# Print the number of missing values in each column
missing_values = df.isnull().sum()
print("Number of missing values in each column:")
print(missing_values)

Number of missing values in each column:
step              4
amount            4
oldbalanceOrig    4
newbalanceOrig    4
oldbalanceDest    4
newbalanceDest    4
isFraud           0
type_CASH_OUT     0
type_DEBIT        0
type_PAYMENT      0
type_TRANSFER     0
type_type         0
dtype: int64


In [4]:
# Handle missing values
df.fillna(0, inplace=True)

# Check and Remove Duplicate Rows

In [5]:
duplicate_rows = df[df.duplicated()]
print("Number of duplicate rows: ", len(duplicate_rows))
print("First few duplicate rows:")
print(duplicate_rows.head(10))

Number of duplicate rows:  546
First few duplicate rows:
        step    amount  oldbalanceOrig  newbalanceOrig  oldbalanceDest  \
11104    7.0   1849.50             0.0             0.0             0.0   
33556    8.0   7759.31             0.0             0.0             0.0   
59969    9.0   2388.93             0.0             0.0             0.0   
60763    9.0  10042.85             0.0             0.0             0.0   
63096    9.0   2783.83             0.0             0.0             0.0   
69138    9.0   6755.57             0.0             0.0             0.0   
72575    9.0   6499.28             0.0             0.0             0.0   
105715  10.0   3237.54             0.0             0.0             0.0   
120246  11.0   2041.31             0.0             0.0             0.0   
131202  11.0   2082.03             0.0             0.0             0.0   

        newbalanceDest isFraud  type_CASH_OUT  type_DEBIT  type_PAYMENT  \
11104              0.0   false          False       F

In [6]:
# Print the number of rows before removing duplicates
print(f"Number of rows before removing duplicates: {len(df)}")

# Remove duplicate rows
df_cleaned = df.drop_duplicates()

# Print the number of rows after removing duplicates
print(f"Number of rows after removing duplicates: {len(df_cleaned)}")

Number of rows before removing duplicates: 6362624
Number of rows after removing duplicates: 6362078


In [7]:
df.drop_duplicates()

,step,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,type_type
0,1.0,9839.64,170136.0,160296.36,0.00,0.00,false,False,False,True,False,False
1,1.0,1864.28,21249.0,19384.72,0.00,0.00,false,False,False,True,False,False
2,1.0,181.00,181.0,0.00,0.00,0.00,true,False,False,False,True,False
3,1.0,181.00,181.0,0.00,21182.00,0.00,true,True,False,False,False,False
4,1.0,11668.14,41554.0,29885.86,0.00,0.00,false,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
6362619,156.0,10221.33,17205.0,6983.67,0.00,0.00,false,False,False,True,False,False
6362620,156.0,5364.42,20100.0,14735.58,0.00,0.00,false,False,False,True,False,False
6362621,156.0,118042.97,52293.0,170335.97,198145.16,80102.20,false,False,False,False,False,False
6362622,156.0,696912.65,20585.0,0.00,0.00,696912.65,false,True,False,False,False,False


# Handle Class Distribution and Encoding

In [8]:
# Print the number of unique classes in the 'isFraud' column
num_classes = df['isFraud'].nunique()
print(f"Number of unique classes in 'isFraud': {num_classes}")

class_distribution = df['isFraud'].value_counts()
print("Class distribution in 'isFraud':\n", class_distribution)

Number of unique classes in 'isFraud': 3
Class distribution in 'isFraud':
 isFraud
false      6354407
true          8213
isFraud          4
Name: count, dtype: int64


In [9]:
# Remove rows where 'isFraud' is neither 'false' nor 'true'
df = df[df['isFraud'].isin([0, 1, 'false', 'true'])]

# Replace 'false' and 'true' strings with numeric values
df['isFraud'] = df['isFraud'].replace({'false': 0, 'true': 1})

# Convert 'isFraud' to numeric
df['isFraud'] = pd.to_numeric(df['isFraud'], errors='coerce')

# Check the distribution
class_distribution = df['isFraud'].value_counts()
print("Updated class distribution in 'isFraud':\n", class_distribution)

Updated class distribution in 'isFraud':
 isFraud
0    6354407
1       8213
Name: count, dtype: int64


/tmp/ipykernel_36/3171151535.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['isFraud'] = df['isFraud'].replace({'false': 0, 'true': 1})


# Resampling and Model Training

In [10]:
X = df.drop('isFraud', axis=1)
y = df['isFraud']

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to balance the classes
X_resampled, y_resampled = smote.fit_resample(X, y)

# Check the new class distribution
print("Class distribution after SMOTE:")
print(pd.Series(y_resampled).value_counts())

Class distribution after SMOTE:
isFraud
0    6354407
1    6354407
Name: count, dtype: int64


# Model Training and Evaluation

In [11]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Train Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5, random_state=42)
dt_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_dt = dt_classifier.predict(X_test)

# Evaluate the model
print("Decision Tree Classifier")
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print(classification_report(y_test, y_pred_dt))

Decision Tree Classifier
Accuracy: 0.9694099503100866
              precision    recall  f1-score   support

           0       0.96      0.98      0.97   1906355
           1       0.98      0.96      0.97   1906290

    accuracy                           0.97   3812645
   macro avg       0.97      0.97      0.97   3812645
weighted avg       0.97      0.97      0.97   3812645



# Stratified Cross-Validation

In [12]:
# Perform Stratified Cross-Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []
fold_class_reports = []

for train_index, test_index in skf.split(X_resampled, y_resampled):
    X_train, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
    y_train, y_test = y_resampled.iloc[train_index], y_resampled.iloc[test_index]
    
    # Train the classifier
    dt_classifier.fit(X_train, y_train)
    
    # Predictions
    y_pred = dt_classifier.predict(X_test)
    
    # Accuracy and classification report for each fold
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    fold_accuracies.append(accuracy)
    fold_class_reports.append(report)

# Print the average accuracy across all folds
print(f"Average Accuracy across folds: {np.mean(fold_accuracies)}")

# Check the classification report for each fold
for idx, report in enumerate(fold_class_reports):
    print(f"Fold {idx + 1} Classification Report:")
    print(report)

Average Accuracy across folds: 0.9694508866207128
Fold 1 Classification Report:
{'0': {'precision': 0.9598812655653953, 'recall': 0.9798714593487042, 'f1-score': 0.9697733576146382, 'support': 1270882}, '1': {'precision': 0.9794433363816222, 'recall': 0.9590457328420207, 'f1-score': 0.9691372184972339, 'support': 1270881}, 'accuracy': 0.9694586001920714, 'macro avg': {'precision': 0.9696623009735088, 'recall': 0.9694585960953624, 'f1-score': 0.9694552880559361, 'support': 2541763}, 'weighted avg': {'precision': 0.9696622971253783, 'recall': 0.9694586001920714, 'f1-score': 0.9694552881810734, 'support': 2541763}}
Fold 2 Classification Report:
{'0': {'precision': 0.9598576726470883, 'recall': 0.9795913389283978, 'f1-score': 0.9696241119174793, 'support': 1270882}, '1': {'precision': 0.9791629343331568, 'recall': 0.9590323562945705, 'f1-score': 0.968993104720321, 'support': 1270881}, 'accuracy': 0.9693118516557209, 'macro avg': {'precision': 0.9695103034901226, 'recall': 0.969311847611484